In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import MeCab
import unicodedata
import re
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import os
import io
import time

In [2]:
# load txt file
def load_def(path):
    # open a txt file as read only
    lines = io.open(path, encoding='UTF-8', errors="ignore").read().strip().split('\n')
    
    return lines

In [7]:
# create each languages list
def create_lang_list(num_example):
    # load txt file
    lines = load_def("dataset/raw.txt")

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_example]]

    return zip(*word_pairs)

In [8]:
# convert unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFC', s)
                    if unicodedata.category(c) != 'Mn')

# preprocess
def preprocess_sentence(w):
    # check japanese lang
    p = re.compile('[\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F]+')
    if p.search(w):
        # Morphological analysis for japanese lang
        m = MeCab.Tagger("-Owakati")
        w = m.parse(w)
    w = unicode_to_ascii(w.lower().strip())
    # create a space between word and the punctuation
    w = re.sub(r"([?!¿.,。])", r" \1 ", w)
    # replacing everything with space except(a-z, A-Z, ".",  "?",  "!",  ",", "-", "ー", , "。", "Kanji", "Katakana", "Hiragana")
    w = re.sub(r"[^a-zA-Z\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F?.!,。¿\-/ {1,}/]+",  " ", w)
    w = w.rstrip().strip()
    
    # add a start and end  token to the sentence
    # model know when to start and end
    w = "<start> " + w + " <end>"
    return w

In [9]:
# check word
en_sentence =u"May I borrow this book?"
ja_sentence = u"プールに行きたい。でも今日は雨."
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(ja_sentence))

<start> may i borrow this book ? <end>
<start> プール に 行き たい  。  でも 今日 は 雨  . <end>


In [10]:
en, ja = create_lang_list(10)
print(en[:10])
print(ja[:10])

('<start> you are back ,  aren t you ,  harold ? <end>', '<start> my opponent is shark . <end>', '<start> this is one thing in exchange for another . <end>', '<start> yeah ,  i m fine . <end>', '<start> don t come to the office anymore .  don t call me either . <end>', '<start> looks beautiful . <end>', '<start> get him out of here ,  because i will fucking kill him . <end>', '<start> you killed him ! <end>', '<start> okay ,  then who ? <end>', '<start> it seems a former employee .  .  . <end>')
('<start> あなた は 戻っ た の ね ハロルド  ? <end>', '<start> 俺 の 相手 は シャーク だ  。 <end>', '<start> 引き換え だ ある 事 と ある 物 の <end>', '<start> もう いい よ ごちそうさま ううん <end>', '<start> もう 会社 に は 来 ない で くれ 電話 も する な <end>', '<start> きれい だ  。 <end>', '<start> 連れ て 行け 殺し そう だ わかっ た か  ? <end>', '<start> 殺し た の か  ! <end>', '<start> わぁ   !  いつも すみません  。  いい の よ   。 <end>', '<start> カンパニー の 元 社員 が <end>')


# Tokenization

In [38]:
def tokenize(lang):
    # victorize a text corpus
    lang_tokenize = tf.keras.preprocessing.text.Tokenizer(filters=' ')
    lang_tokenize.fit_on_texts(lang)
    # to sequence of integer  e.g. ['こんにちは 今日　は', "today is so cold"] →[[1, 2], [3, 4, 5, 6]]
    tensor = lang_tokenize.texts_to_sequences(lang)
    print(tensor)
    # Fixed length because length of sequence of integers are different e.g. [[1, 2], [3, 4, 5, 6]] 
    # → →[[1 2 0 0] [3 4 5 6]]
    # return (len(sequences), maxlen) 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                                                                         padding="post")
    print(tensor)
    return tensor, lang_tokenize

In [40]:
# example
tokenize(['こんにちは 今日　は', "today is so cold"])

[[1, 2], [3, 4, 5, 6]]
[[1 2 0 0]
 [3 4 5 6]]


(array([[1, 2, 0, 0],
        [3, 4, 5, 6]], dtype=int32),
 <keras_preprocessing.text.Tokenizer at 0x1a3793af10>)